# N-gram Predictor: Build Your Own Next-Word Prediction Tool

This notebook guides you through creating a simple next-word prediction system using unigrams and bigrams.
You will learn how to process text, build n-gram models, and predict upcoming words based on context.

## Step 1: Text Preprocessing

Before building our model, we need to clean and tokenize the training text. This involves converting text to lowercase, removing punctuation, and splitting into words.

In [ ]:
import re

# Example training text
training_text = "I love cats. I love dogs. Cats love fish. Dogs love bones."

# Function for simple text preprocessing

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r"[^a-z\s]", "", text)
    # Tokenize into words
    tokens = text.split()
    return tokens

# Preprocess the training text
tokens = preprocess_text(training_text)
print(tokens)

## Step 2: Extract N-grams (Unigrams and Bigrams)

Next, we will build dictionaries to count unigrams and bigrams from the tokens. These counts will help us estimate probabilities later.

In [ ]:
from collections import defaultdict

# Initialize dictionaries for counts
unigram_counts = defaultdict(int)
bigram_counts = defaultdict(int)

# Build unigram counts
for word in tokens:
    unigram_counts[word] += 1

# Build bigram counts
for i in range(len(tokens) - 1):
    bigram = (tokens[i], tokens[i+1])
    bigram_counts[bigram] += 1

# Display counts
print("Unigram counts:")
print(dict(unigram_counts))
print("\nBigram counts:")
print(dict(bigram_counts))

## Step 3: Calculate Probabilities

Using the counts, we can estimate the probability of a word given a context (for bigrams) or just the probability of a word (for unigrams).
For smoothing unseen n-grams, we can add a small constant (like 1) to all counts, but here we'll start with basic probability calculation.

In [ ]:
def calculate_unigram_probability(word):
    total_unigrams = sum(unigram_counts.values())
    return unigram_counts[word] / total_unigrams if word in unigram_counts else 0

# Example
print(f"Probability of 'cats': {calculate_unigram_probability('cats')}")

## Step 4: Predict Next Word

Given a context (like a single word or phrase), we want to predict the next word. We'll look at bigram counts to find the most probable next words.

In [ ]:
def predict_next_word(context, top_k=3):
    # Preprocess the context
    context = context.lower()
    tokens_ctx = preprocess_text(context)
    if not tokens_ctx:
        return []
    last_word = tokens_ctx[-1]
    # Gather predictions
    candidates = {}
    total_bigrams_with_prefix = 0
    for (w1, w2), count in bigram_counts.items():
        if w1 == last_word:
            candidates[w2] = count
            total_bigrams_with_prefix += count
    # Calculate probabilities
    predictions = []
    for word, count in candidates.items():
        prob = count / total_bigrams_with_prefix if total_bigrams_with_prefix > 0 else 0
        predictions.append((word, prob))
    # Sort by probability
    predictions.sort(key=lambda x: x[1], reverse=True)
    return predictions[:top_k]

# Example prediction
predicted = predict_next_word("I love")
print("Top predictions for 'I love':")
for word, prob in predicted:
    print(f"{word} ({prob:.2f})")

## Additional: Handling Unseen Contexts with Smoothing

To improve predictions for unseen bigrams, we can apply smoothing techniques like add-one smoothing. This adjusts counts to avoid zero probabilities.

In [ ]:
def predict_next_word_smoothed(context, top_k=3, alpha=1):
    # Preprocess the context
    context = context.lower()
    tokens_ctx = preprocess_text(context)
    if not tokens_ctx:
        return []
    last_word = tokens_ctx[-1]
    # Gather candidates with smoothing
    candidates = {}
    total_bigrams_with_prefix = 0
    for (w1, w2), count in bigram_counts.items():
        if w1 == last_word:
            candidates[w2] = count + alpha
    # Add fake counts for unseen words
    all_words = set(unigram_counts.keys())
    for word in all_words:
        if word not in candidates:
            candidates[word] = alpha
    total_bigrams_with_prefix = sum(candidates.values())
    # Calculate probabilities
    predictions = []
    for word, count in candidates.items():
        prob = count / total_bigrams_with_prefix
        predictions.append((word, prob))
    # Sort and return top-k
    predictions.sort(key=lambda x: x[1], reverse=True)
    return predictions[:top_k]

# Example with smoothing
predicted_smooth = predict_next_word_smoothed("I love")
print("Top smoothed predictions for 'I love':")
for word, prob in predicted_smooth:
    print(f"{word} ({prob:.2f})")